<a href="https://colab.research.google.com/github/akihiros/ngrams/blob/master/Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ベクトル空間モデル

- 文書集合のベクトル空間モデルを作成し、クエリに対して類似度計算する

In [6]:
!pip install janome

     |████████████████████████████████| 21.5MB 107kB/s 


## 演習10.1 文字列抽出

In [0]:
# n-gramに分割する関数：nで指定
def n_gram(target, n):
  return [target[idx:idx + n] for idx in range(len(target) - n + 1)]

In [8]:
# 単語単位で3-gramに分割
from janome.tokenizer import Tokenizer

t = Tokenizer(wakati=True)
s = "兎や、キツネのほかに、イタチの足あと、ネズミの足あと、猫の足あと、みんなちがう。"
words = t.tokenize(s)

print(n_gram(words, 3))

[['兎', 'や', '、'], ['や', '、', 'キツネ'], ['、', 'キツネ', 'の'], ['キツネ', 'の', 'ほか'], ['の', 'ほか', 'に'], ['ほか', 'に', '、'], ['に', '、', 'イタチ'], ['、', 'イタチ', 'の'], ['イタチ', 'の', '足'], ['の', '足', 'あと'], ['足', 'あと', '、'], ['あと', '、', 'ネズミ'], ['、', 'ネズミ', 'の'], ['ネズミ', 'の', '足'], ['の', '足', 'あと'], ['足', 'あと', '、'], ['あと', '、', '猫'], ['、', '猫', 'の'], ['猫', 'の', '足'], ['の', '足', 'あと'], ['足', 'あと', '、'], ['あと', '、', 'みんな'], ['、', 'みんな', 'ちがう'], ['みんな', 'ちがう', '。']]


In [12]:
# 文字単位で3-gramに分割
chars = list(s)
print(n_gram(chars, 3))

[['兎', 'や', '、'], ['や', '、', 'キ'], ['、', 'キ', 'ツ'], ['キ', 'ツ', 'ネ'], ['ツ', 'ネ', 'の'], ['ネ', 'の', 'ほ'], ['の', 'ほ', 'か'], ['ほ', 'か', 'に'], ['か', 'に', '、'], ['に', '、', 'イ'], ['、', 'イ', 'タ'], ['イ', 'タ', 'チ'], ['タ', 'チ', 'の'], ['チ', 'の', '足'], ['の', '足', 'あ'], ['足', 'あ', 'と'], ['あ', 'と', '、'], ['と', '、', 'ネ'], ['、', 'ネ', 'ズ'], ['ネ', 'ズ', 'ミ'], ['ズ', 'ミ', 'の'], ['ミ', 'の', '足'], ['の', '足', 'あ'], ['足', 'あ', 'と'], ['あ', 'と', '、'], ['と', '、', '猫'], ['、', '猫', 'の'], ['猫', 'の', '足'], ['の', '足', 'あ'], ['足', 'あ', 'と'], ['あ', 'と', '、'], ['と', '、', 'み'], ['、', 'み', 'ん'], ['み', 'ん', 'な'], ['ん', 'な', 'ち'], ['な', 'ち', 'が'], ['ち', 'が', 'う'], ['が', 'う', '。']]


## 演習10.2 クエリに対する類似度計算

In [0]:
# 文書集合
doc = ['フィギュアスケートのグランプリファイナルは、カナダのバンクーバーで開幕し、女子ショートプログラムで、紀平梨花が首位に立った。',
       'フィギュアのグランプリファイナルは、宇野昌磨は２位発進となった。',
       'フィギュアスケート男子の羽生結弦の年内復帰が難しくなった。日本スケート連盟はカナダのグランプリファイナル欠場を発表した。',
       'フィギュア男子',
       'スケート女子']

In [85]:
import numpy as np
from janome.tokenizer import Tokenizer
 
t = Tokenizer()

docs = np.array([])

for i in range(len(doc)):
    tokens = t.tokenize(doc[i])
    noun_list = np.array([])

    for token in tokens:
        # 品詞を取り出し
        partOfSpeech = token.part_of_speech.split(',')[0]
    
        if partOfSpeech == u'名詞':
            noun_list = np.append(noun_list, token.surface)

    noun_list = ' '.join(noun_list)  # 名詞のリスト
    docs = np.append(docs, noun_list)  # 文書数×名詞のリスト

print(docs)

['フィギュア スケート グランプリ ファイナル カナダ バンクーバー 開幕 女子 ショートプログラム 紀平 梨花 首位'
 'フィギュア グランプリ ファイナル 宇野 昌 磨 ２ 位 発進'
 'フィギュア スケート 男子 羽生 結 弦 年内 復帰 日本 スケート 連盟 カナダ グランプリ ファイナル 欠場 発表' 'フィギュア 男子'
 'スケート 女子']


In [86]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
 
np.set_printoptions(precision=2)
 
vectorizer = TfidfVectorizer(use_idf=True, token_pattern=u'(?u)\\b\\w+\\b')
vecs = vectorizer.fit_transform(docs)
 
print(vecs.toarray())

[[0.27 0.22 0.33 0.22 0.33 0.22 0.19 0.   0.27 0.   0.   0.   0.   0.
  0.   0.33 0.   0.   0.   0.   0.   0.33 0.   0.   0.   0.33 0.33 0.  ]
 [0.   0.25 0.   0.   0.   0.25 0.21 0.37 0.   0.37 0.   0.   0.   0.
  0.37 0.   0.   0.   0.   0.37 0.37 0.   0.   0.   0.   0.   0.   0.37]
 [0.22 0.18 0.   0.37 0.   0.18 0.15 0.   0.   0.   0.27 0.27 0.27 0.27
  0.   0.   0.27 0.22 0.27 0.   0.   0.   0.27 0.27 0.27 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.57 0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.82 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.64 0.   0.   0.   0.   0.77 0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]]


In [87]:
for k,v in sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1]):
    print(k,v)

カナダ 0
グランプリ 1
ショートプログラム 2
スケート 3
バンクーバー 4
ファイナル 5
フィギュア 6
位 7
女子 8
宇野 9
年内 10
弦 11
復帰 12
日本 13
昌 14
梨花 15
欠場 16
男子 17
発表 18
発進 19
磨 20
紀平 21
結 22
羽生 23
連盟 24
開幕 25
首位 26
２ 27


In [118]:
# コサイン類似度
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

#TODO:クエリから名詞を抜き出して、kを検索して一致すればvを抜き出す
#TODO:vと一致するvecs.toarray()TF-IDFを検索し、Yとする
#TODO:Xは上記同様クエリに対するTF-IDFを計算する

# 面倒なので手打ちする
delta = 1e-7

X = np.array([0.57, 0.82])  # フィギュア男子：6, 17
D1 = np.array([0.19, 0.])  #doc1の重み
D2 = np.array([0.21, 0.])
D3 = np.array([0.15, 0.22])
print(np.dot(X, D1), np.dot(X, D2), np.dot(X, D3))  # 内積計算
# print(cos_sim(X, D1), cos_sim(X, D2), cos_sim(X, D3))

X = np.array([0.64, 0.77])  # スケート女子
D1 = np.array([0.22, 0.27])
D2 = np.array([0., 0.])
D3 = np.array([0.37, 0.])
print(np.dot(X, D1), np.dot(X, D2), np.dot(X, D3))
# print(cos_sim(X, D1), cos_sim(X, D2), cos_sim(X, D3))

0.1083 0.11969999999999999 0.26589999999999997
0.3487 0.0 0.2368


In [116]:
# 例題の再現が微妙。おそらく明示されていない小数点の影響。
X = np.array([0.35, 0.94])
D1 = np.array([0., 0.])
D2 = np.array([0.18, 0.])
D3 = np.array([0.16, 0.43])
print(np.dot(X, D1), np.dot(X, D2), np.dot(X, D3))

0.0 0.063 0.46019999999999994
